In [1]:
!python -m ipykernel install --user --name=myenv

Installed kernelspec myenv in /home/douliyang/.local/share/jupyter/kernels/myenv


In [1]:
from iree import compiler as ireec

In [2]:
# Compile a module.
INPUT_MLIR = """
module @arithmetic {
  func.func @simple_mul(%arg0: tensor<4xf32>, %arg1: tensor<4xf32>) -> tensor<4xf32> {
    %0 = arith.mulf %arg0, %arg1 : tensor<4xf32>
    return %0 : tensor<4xf32>
  }
}
"""

In [3]:
# Compile using the CUDA backend
compiled_flatbuffer = ireec.tools.compile_str(
    INPUT_MLIR,
    target_backends=["cuda"]  # Specify CUDA backend
)

In [4]:
from iree import runtime as ireert
import numpy as np

In [6]:
# Register the module with a runtime context.
# Use the "cuda" driver for executing on NVIDIA GPU
config = ireert.Config("cuda")  # 指定使用 CUDA 驱动
ctx = ireert.SystemContext(config=config)

# Load the compiled flatbuffer (compiled with the CUDA backend)
vm_module = ireert.VmModule.copy_buffer(ctx.instance, compiled_flatbuffer)
ctx.add_vm_module(vm_module)

# Invoke the function and print the result
print("INVOKE simple_mul")
arg0 = np.array([1., 2., 3., 4.], dtype=np.float32)  # 输入张量 1
arg1 = np.array([4., 5., 6., 7.], dtype=np.float32)  # 输入张量 2
f = ctx.modules.arithmetic["simple_mul"]  # 调用 simple_mul 函数
results = f(arg0, arg1).to_host()  # 执行并获取结果
print("Results:", results)

INVOKE simple_mul
Results: [ 4. 10. 18. 28.]
